#  1）建立对象与导入原始数据

In [1]:
######建立对象
import sys
sys.path.append('E:\\python-dq\\py\\work')
import imp

import dq_work.goods.goods_1 as goods


imp.reload(goods)#再次引入模块，默认只有一次，调试程序时有用

#建立对象
###建立对象的相关参数变量

company='上海'
date_Range='202207'
date_Range_all='20220701-0731'
in_file_path='e:/python-dq/work_data/goods_data/in/'
out_file_path='e:/python-dq/work_data/goods_data/out/'
public_path='e:/python-dq/work_data/public_data/out/'

all_sku_goods_obj=goods.all_sku_goods(company,date_Range,date_Range_all,in_file_path,out_file_path,public_path)


In [2]:
#导入原始数据
#需要准备的导入文件清单
######################################

in_file1='03_业务报表-销售报表-销售-'+company+ '-'+date_Range_all+'.csv'
in_file2='01-用劵分摊-商品层级-原版-'+company+ '-all-'+date_Range_all+'.csv'
in_file3='A-分析-门店基础资料-'+company+'.csv'
in_file4='采购收入_'+company+ '_'+date_Range_all+'.csv'
in_file5='B-分析-品类基础资料.csv'
######################################

df_原始销售_all,df_原始用券_all,df_采购收入原始数据,df_类别,df_商品主档= \
all_sku_goods_obj.load_data_all_sku(in_file1,in_file2,in_file3,in_file4,in_file5)


导入all sku 原始数据 over


# 2）计算销售类数据

In [3]:
########计算########
#合并sku的销售数据与券数据
df_sale_coupons=all_sku_goods_obj.sale_coupons_join_all_sku(df_原始销售_all,df_原始用券_all)

all sku的销售数据与券数据合并完成
销售净额    1.125489e+08
dtype: float64
销售净额    1.125489e+08
dtype: float64
万家承担金额（去税）    7.308988e+06
dtype: float64


In [4]:
#正数销售参与oi分摊
bool_1=df_原始销售_all['销售净额'] > 0
df_原始销售_all_正销售=df_原始销售_all.loc[bool_1]
##分摊oi
df_oi1,df_oi2=all_sku_goods_obj.all_sku_oi_share(df_原始销售_all_正销售,df_采购收入原始数据)
df_oi3=df_oi1.append(df_oi2)

金额    6154987.87
dtype: float64
OI分摊金额    6317209.46
dtype: float64
OI分摊金额   -162221.59
dtype: float64
all sku 的OI 分摊计算完成


In [5]:
#合并处理销售、券、oi并输出结果df
df_all1=all_sku_goods_obj.all_sku_merge_out_df(df_sale_coupons,df_oi3)

print(df_all1.loc[:,['OI分摊金额']].sum())  
print(df_all1.loc[:,['销售净额']].sum()) 




df合并处理finish
OI分摊金额    6154987.87
dtype: float64
销售净额    1.125489e+08
dtype: float64


# 3）库存周转相关数据计算

In [6]:
#导入原始数据
#需要准备的导入文件清单
######################################
in_file1='03_业务报表-销售报表-库存-'+company+ '-'+date_Range_all+'.csv'
######################################

df_原始库存= \
all_sku_goods_obj.load_data_stock_all_sku(in_file1)


导入原始库存数据 over
库存金额    6.113569e+09
dtype: float64


In [7]:
########计算########
#计算库存数据
df_stock1=all_sku_goods_obj.stock_compute_all_sku(df_原始库存)


库存计算finish


# 4）销售与库存合并，导出到mysql

In [9]:
#销售df与库存df的合并
###销售df，去上述的精加工后的销售毛利df--df_out
df_sale_stock=all_sku_goods_obj.merge_sale_stock_all_sku(df_all1,df_stock1)

#添加库存原始数据中的sku主档信息
#df_商品主档_stock=goods_obj.create_sku_master_data_stock(df_商品主档,df_原始库存)
#精加工输出的DF
df_out2=all_sku_goods_obj.finish_machining_outfile_all_sku(df_sale_stock,df_商品主档,df_类别)

销售与库存合并处理finish
精加工 dataframe over


In [19]:
#将结果落库mysql
#落库的table
in_table1='all_sku_' + date_Range
all_sku_goods_obj.input_mysql_all_sku(df_out2,in_table1)


all sku input mysql


# 5) 过滤掉异常的goods单品

In [19]:
#选择正库存
bool_2=df_out2['库存金额'] > 0
df_out2_正库存=df_out2.loc[bool_2]
#过滤掉异常goods
df_anormal_毛利率=all_sku_goods_obj.find_anormal(df_out2_正库存,'总毛利率')
df_anormal_周转=all_sku_goods_obj.find_anormal(df_out2_正库存,'年周转次数')
df_anormal_销售净额=all_sku_goods_obj.find_anormal(df_out2_正库存,'销售净额')

#过滤掉负库存
bool_3=df_out2['库存金额'] < 0
df_out2_负库存=df_out2.loc[bool_3]

#合并异常的df
df_anormal_1=df_anormal_毛利率.loc[:,[ '商品编码']]
df_anormal_2=df_anormal_周转.loc[:,[ '商品编码']]
df_anormal_3=df_out2_负库存.loc[:,[ '商品编码']]
df_anormal_4=df_anormal_销售净额.loc[:,[ '商品编码']]
df_anormal =df_anormal_1.append([df_anormal_2,df_anormal_3,df_anormal_4])
df_anormal.drop_duplicates(subset = ['商品编码'],keep='first',inplace=True)




异常数据计算 finish，返回df
异常数据计算 finish，返回df
异常数据计算 finish，返回df


In [20]:
#将anormalde goods结果落库mysql
#落库的table
in_table2='all_sku_anormal' + date_Range
all_sku_goods_obj.input_mysql_all_sku(df_anormal,in_table2)

all sku input mysql


# 单品贡献率计算over

# 6）将sku对应供应商的数据落库mysql

In [13]:
import pandas as pd
# 获得sku对应供应商编码与名称
df_原始销售_all.columns

g_sku_supplier =df_原始销售_all.groupby(['商品编码','供应商编码','供应商名称'])
df_sku_supplier_xs=pd.DataFrame(g_sku_supplier[['销售净额']].sum())
df_sku_supplier_xs.reset_index(inplace=True)

xz_idx =df_sku_supplier_xs.groupby(['商品编码'])['销售净额'].idxmax()

df_sku_supplier=df_sku_supplier_xs.loc[xz_idx]
df_sku_supplier.drop_duplicates(subset = ['商品编码'],keep='first',inplace=True)

In [14]:
#将sku的对应供应商编码名称结果落库mysql
#落库的table
in_table3='sku_supplier' + date_Range
all_sku_goods_obj.input_mysql_all_sku(df_sku_supplier,in_table3)

all sku input mysql
